# Configuration

| Supplementing | Converting | Scaling  |
| ------------- | ---------- | -------- |
| knn / mode    | binary     | standard |


In [10]:
# change position of working directory
import os
import sys

module_path = os.path.abspath("..")
if module_path not in sys.path:
    sys.path.append(module_path)

In [11]:
# import all necessary packages
import pandas as pd
import numpy as np
from sklearn.ensemble import VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression

# import all internal packages
from preprocessing.supplementing.complex.by_knn import by_knn
from preprocessing.supplementing.simple.by_mode import by_mode
from preprocessing.converting.to_binary_columns import to_binary_columns
from preprocessing.scaling.standard import standard
from cross_validation import stratified_cross_validation, validate


In [12]:
# read data
db_path = "../../db/credit-card-approval/credit_card_approval.csv"
df = pd.read_csv(db_path)


In [13]:
# fill in the missing data
by_mode(
    df,
    [
        "CODE_GENDER",
        "FLAG_OWN_CAR",
        "FLAG_OWN_REALTY",
        "CNT_CHILDREN",
        "NAME_EDUCATION_TYPE",
        "NAME_FAMILY_STATUS",
        "NAME_HOUSING_TYPE",
        "JOB",
        "STATUS",
        "FLAG_MOBIL",
        "FLAG_WORK_PHONE",
        "FLAG_PHONE",
        "FLAG_EMAIL",
    ],
)
by_knn(df, ["AMT_INCOME_TOTAL", "DAYS_BIRTH", "DAYS_EMPLOYED", "BEGIN_MONTHS"])


,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,JOB,BEGIN_MONTHS,STATUS,TARGET
0,5046172,M,Y,N,No children,180000.0,Secondary / secondary special,Single / not married,House / apartment,-20142.0,-108.0,1.0,0.0,0.0,0.0,High skill tech staff,-24.0,0,0
1,5068809,M,Y,Y,1 children,112500.0,Higher education,Married,House / apartment,-11809.0,-536.0,1.0,0.0,0.0,1.0,Accountants,0.0,X,0
2,5067957,M,Y,Y,No children,360000.0,Higher education,Married,House / apartment,-22111.0,-3854.0,1.0,0.0,0.0,0.0,Managers,-40.0,X,0
3,5146339,F,N,Y,No children,202500.0,Secondary / secondary special,Married,House / apartment,-16173.0,-674.0,1.0,0.0,0.0,0.0,Sales staff,-2.0,C,0
4,5118164,M,Y,N,2+ children,225000.0,Secondary / secondary special,Married,House / apartment,-12801.0,-4357.0,1.0,0.0,0.0,0.0,Laborers,-27.0,X,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53762,5029349,M,Y,Y,No children,270000.0,Higher education,Married,House / apartment,-10942.0,-412.0,1.0,0.0,0.0,0.0,Drivers,-43.0,0,0
53763,5090719,F,N,N,No children,234000.0,Secondary / secondary special,Widow,House / apartment,-21932.0,-10232.0,1.0,1.0,0.0,0.0,Laborers,-35.0,0,0
53764,5023812,F,N,Y,1 children,202500.0,Secondary / secondary special,Separated,House / apartment,-13956.0,-5429.0,1.0,0.0,0.0,0.0,Laborers,-15.0,C,0
53765,5099871,F,Y,Y,2+ children,112500.0,Secondary / secondary special,Married,House / apartment,-9994.0,-644.0,1.0,1.0,0.0,0.0,Sales staff,-4.0,C,0


In [14]:
# processing of categorical data
df["CODE_GENDER"].replace({"F": 0, "M": 1}, inplace=True)
df["FLAG_OWN_CAR"].replace({"N": 0, "Y": 1}, inplace=True)
df["FLAG_OWN_REALTY"].replace({"N": 0, "Y": 1}, inplace=True)
df = to_binary_columns(
    df,
    [
        "CNT_CHILDREN",
        "NAME_EDUCATION_TYPE",
        "NAME_FAMILY_STATUS",
        "NAME_HOUSING_TYPE",
        "JOB",
        "STATUS",
    ],
)[0]

In [15]:
# assign to the dataframe
df["AMT_INCOME_TOTAL"] = standard(df["AMT_INCOME_TOTAL"])
df["DAYS_BIRTH"] = standard(df["DAYS_BIRTH"])
df["DAYS_EMPLOYED"] = standard(df["DAYS_EMPLOYED"])
df["BEGIN_MONTHS"] = standard(df["BEGIN_MONTHS"])

# display
df[["AMT_INCOME_TOTAL", "DAYS_BIRTH", "DAYS_EMPLOYED", "BEGIN_MONTHS"]]


,AMT_INCOME_TOTAL,DAYS_BIRTH,DAYS_EMPLOYED,BEGIN_MONTHS
0,-0.167635,-1.505930,1.112060,-0.345254
1,-0.814446,0.937664,0.932457,1.373636
2,1.557194,-2.083325,-0.459885,-1.491181
3,0.047969,-0.342048,0.874547,1.230395
4,0.263572,0.646767,-0.670960,-0.560115
...,...,...,...,...
53762,0.694780,1.191906,0.984491,-1.706042
53763,0.349814,-2.030835,-3.136303,-1.133079
53764,0.047969,0.308071,-1.120807,0.299330
53765,-0.814446,1.469900,0.887136,1.087154


In [16]:
# remove unnecessary columns
df.drop("FLAG_WORK_PHONE", axis=1, inplace=True)
df.drop("FLAG_MOBIL", axis=1, inplace=True)
df.drop("ID", axis=1, inplace=True)
df.drop("BEGIN_MONTHS", axis=1, inplace=True)
df.drop("FLAG_PHONE", axis=1, inplace=True)

In [17]:
# Save model data
prepared_db_path = "../../db/models-data/gamma.csv"
df.to_csv(prepared_db_path, index=False)


# Training

In [18]:
# prepare classifiers
xgb = XGBClassifier(eval_metric="logloss")
dtc = DecisionTreeClassifier()

# prepare cross-validation data
train_test_splits = stratified_cross_validation(df, "TARGET", n_folds=10)

In [19]:
# create voting classifier
voting_clf = VotingClassifier(estimators=[("xgb", xgb), ("dtc", dtc)], voting="hard")

# validate
np.mean(validate(train_test_splits, voting_clf, "TARGET"))

1.0

In [20]:
# create stacking classifier
stacking_clf = StackingClassifier(
    estimators=[("xgb", xgb), ("dtc", dtc)], final_estimator=LogisticRegression()
)

# validate
np.mean(validate(train_test_splits, stacking_clf, "TARGET"))

1.0